# Setup

In [14]:
# Installs

%pip install -qU langchain-community langchain-core==0.2.40 langchain_experimental langchain-google-community langchain-google-genai langchain-huggingface==0.0.3 langchain-openai langchain-qdrant langchain-google-vertexai
%pip install -qU -q chainlit==1.1.302
%pip install -qU -q docx2txt
%pip install -qU google-cloud-aiplatform
%pip install -qU google-cloud-discoveryengine
%pip install -qU nltk
%pip install -qU openpyxl
%pip install -qU pymupdf
%pip install -qU python-dotenv
%pip install -qU ragas==0.1.20
%pip install -qU tqdm

# Verify installed packages have compatible dependencies
%pip check

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
No broken requirements found.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Download punkt_tab module that is used for sentence tokenizaiton

import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/david/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Environment Variables

In [16]:
# Get environment variables

import os
from dotenv import load_dotenv
import uuid

# Load environment variables from .env file
load_dotenv()

True

# Google Auth

In [17]:
# Set Google user permissions

import os
import sys

# If running in Colab, use the permissions of the currently authenticated user
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

# # If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file 
# else:
#     os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'credentials.json'

#####

def test_google_perms():
    from google.cloud import storage

    # Now, you can use the Google Cloud client libraries
    client = storage.Client()

    # List all buckets in your project
    buckets = list(client.list_buckets())
    print(buckets)

test_google_perms()

[<Bucket: 395640738565_us_import_content_with_faq_csv>, <Bucket: aie4-demo-docs>]


In [18]:
# Initialize Vertex AI

import vertexai

vertexai.init(project=os.environ['PROJECT_ID'], location=os.environ['REGION'])

In [19]:
# Verfify that our Google API key works

import requests

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}'
  
headers = {
    "Content-Type": "application/json",
}
  
data = {"contents":[{"parts":[{"text":"Explain how AI works"}]}]}

response = requests.post(url, headers=headers, json=data)
print(response.text)

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "## How AI Works: A Simplified Explanation\n\nArtificial intelligence (AI) is a broad field that encompasses many techniques and approaches. Here's a simplified explanation, focusing on two main categories:\n\n**1. Machine Learning (ML):**\n\n* **The core principle:**  ML algorithms learn from data. They are trained on massive datasets, identifying patterns and relationships within the data. \n* **Example:** If you train an image recognition system on thousands of pictures of cats and dogs, it learns to identify key features that differentiate them.  Then, when shown a new picture, it can predict whether it's a cat or a dog based on the learned patterns.\n* **Types of ML:**\n    * **Supervised Learning:**  The algorithm is given labeled data (e.g., images of cats labeled \"cat\"). It learns to predict the label for unseen data.\n    * **Unsupervised Learning:** The algorithm is given unlabel

# Utils

In [20]:
# Load the docs

# TODO - add TextLoader refernece
# TODO - add CSVLoader reference
# https://python.langchain.com/docs/integrations/document_loaders/pymupdf/
# https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import Docx2txtLoader

def process_file(path: str):

    docs = None

    # Select the right loader
    if 'txt' in path.lower():
        loader = TextLoader(path)
    elif 'pdf' in path.lower():
        loader = PyMuPDFLoader(path)
    elif 'docx' in path.lower():
        loader = Docx2txtLoader(path)
    else:
        print(f'No document loader found for {path}')

    docs = loader.load()

    return docs

#####

def test_process_file():
    doc = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    print(doc[0].metadata)

test_process_file()

{'source': 'docs/CHW_EOC_04-21-2017_ENG.pdf', 'file_path': 'docs/CHW_EOC_04-21-2017_ENG.pdf', 'page': 0, 'total_pages': 134, 'format': 'PDF 1.5', 'title': '', 'author': 'PTN Consulting', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2013', 'producer': 'Microsoft® Word 2013', 'creationDate': "D:20170421101839-07'00'", 'modDate': "D:20170421103401-07'00'", 'trapped': ''}


In [21]:
# Create embeddings using Hugging Face 

from langchain_huggingface import HuggingFaceEmbeddings

def create_embeddings_huggingface(model='Snowflake/snowflake-arctic-embed-m') -> HuggingFaceEmbeddings:

    # Initialize the OpenAIEmbeddings class
    embeddings = HuggingFaceEmbeddings(model_name=model)

    return embeddings

#####

def test_create_embeddings_huggingface():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_huggingface()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_huggingface()

[-0.018624262884259224, 0.07942049950361252, -0.01396368071436882, -0.019285650923848152, 0.019548555836081505, 0.04386259615421295, 0.001751587842591107, 0.003699950175359845, -0.035668954253196716, -0.03971784934401512, -0.012464510276913643, 0.020438581705093384, 0.017930110916495323, -0.0047818077728152275, -0.033828254789114, -0.010549304075539112, 0.06566156446933746, -0.01852135919034481, -0.024217918515205383, -0.028709201142191887, -0.02063765749335289, -0.01962468773126602, -0.004117606673389673, 0.03691602125763893, -0.023654839023947716, 0.03305622190237045, 0.03593701496720314, 0.05134712532162666, -0.07007365673780441, -0.0522121861577034, 0.029391692951321602, 0.03666972368955612, -0.017945537343621254, -0.06466532498598099, -0.018098600208759308, 0.007805246859788895, -0.0360274501144886, -0.03589756786823273, -0.0524902306497097, -0.00038497804780490696, -0.002180682262405753, 0.004602802451699972, 0.012346711941063404, 0.03780534490942955, -0.028559066355228424, -0.01

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='Snowflake/snowflake-arctic-embed-m', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model='text-embedding-ada-002') -> OpenAIEmbeddings:

    # Initialize the OpenAIEmbeddings class
    embeddings = OpenAIEmbeddings(model=model)

    return embeddings

#####

def test_create_embeddings_openai():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_openai()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_openai()

[-0.013160358183085918, -0.003683689283207059, 0.043114304542541504, -0.04365256056189537, -0.03256448730826378, 0.013981198891997337, -0.025338400155305862, -0.019888557493686676, -0.016336068511009216, 0.0030848796013742685, -0.0008982146391645074, 0.010583457536995411, 0.0036096791736781597, 0.004087381064891815, -0.00582325691357255, -0.0022993620950728655, 0.051349617540836334, -0.009251274168491364, 0.005369103047996759, 0.0052917287684977055, -0.013860090635716915, 0.0024625209625810385, 0.003858622396364808, -0.00100334279704839, -0.01402156800031662, 0.017977749928832054, 0.01059691421687603, -0.01219822559505701, -0.021785909309983253, -0.021691715344786644, 0.036520667374134064, 0.01644372008740902, -0.009547315537929535, -0.008551541715860367, -0.01217804104089737, 0.022526012733578682, 0.0001328819344053045, -0.00982317142188549, 0.022727858275175095, -0.00042955350363627076, 0.009109982289373875, 0.0032043049577623606, -0.001017640228383243, 0.004864488262683153, -0.00685

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x33e62e8d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x33e64e050>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [23]:
# Create embeddings using Vertex AI

# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/

from langchain_google_vertexai import VertexAIEmbeddings

def create_embeddings_vertexai(model="text-embedding-004") -> VertexAIEmbeddings:

    # Initialize the VertexAIEmbeddings class
    embeddings = VertexAIEmbeddings(model_name='text-embedding-005')

    return embeddings

#####

def test_create_embeddings_vertexai():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_vertexai()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_vertexai()

[-0.006481274031102657, 0.0586259625852108, -0.03288882598280907, -0.03636451065540314, -0.0024562713224440813, 0.021607939153909683, -0.02025117725133896, 0.026645563542842865, -0.03338305279612541, 0.01319699827581644, -0.023710399866104126, -0.00839491281658411, 5.9321864682715386e-05, -0.04143621399998665, -0.030560482293367386, 0.02604156918823719, 0.023944251239299774, -0.06115581467747688, -0.022342074662446976, 0.0035691543016582727, 0.03404016047716141, 0.0004553132166620344, -0.032472215592861176, -0.03381289914250374, -0.0003450660442467779, -0.025263842195272446, 0.03621809929609299, -0.004921380430459976, -0.05756421759724617, 0.03279789537191391, 0.0391770675778389, 0.010217544622719288, 0.07010894268751144, 0.06245170161128044, 0.057976678013801575, 0.026232847943902016, 0.02897932194173336, 0.008729349821805954, -0.00454883836209774, -0.05016933009028435, 0.01923929899930954, -0.04184930771589279, -0.09402519464492798, -0.013684693723917007, 0.016147518530488014, 0.0347

VertexAIEmbeddings(client=<vertexai.language_models.TextEmbeddingModel object at 0x33c139710>, async_client=None, project='aie4-demo', location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='text-embedding-005', full_model_name=None, client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, api_endpoint=None, api_transport=None, default_metadata=(), additional_headers=None, client_cert_source=None, credentials=None, client_preview=None, temperature=None, max_output_tokens=None, top_p=None, top_k=None, n=1, streaming=False, model_family=None, safety_settings=None, tuned_model_name=None, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_good_batch_size': 5, 'lock': <unlocked _thread.lock object at 0x3075edc

In [24]:
# Remove empty chunks 

def remove_empty_chunks(chunks_start: list) -> list:
    
    start = len(chunks_start)
    # print(f'start - {start} chunks')
    
    # Remove empty chunks
    chunks_end = [chunk for chunk in chunks_start if chunk.page_content.strip()]

    end = len(chunks_end)
    # print(f'end - {end} chunks')

    return chunks_end   

#####

from langchain.schema import Document

def test_remove_empty_chunks():
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')

    # Added a test doc
    doc = Document(
        page_content='',
        metadata=docs[0].metadata
    )
    docs.append(doc)
    print(len(docs))

    # Remove the empty doc (chunk)
    docs = remove_empty_chunks(docs)
    print(len(docs))

test_remove_empty_chunks()

135
134


In [25]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(documents: list, chunk_size=500, chunk_overlap=50) -> list:

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    chunks_start = text_splitter.split_documents(documents)

    chunks_end = remove_empty_chunks(chunks_start=chunks_start)

    return chunks_end

#####

def test_chunk_docs_recursive(): 
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    chunks = chunk_docs_recursive(documents=docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_recursive()

615
Medi-Cal
Member Handbook
Combined Evidence of Coverage
and Disclosure Form                   
 
CAHealthWellness.com
For TTY, contact California Relay by dialing 711 and 
provide the Member Services number: 1-877-658-0305


In [26]:
# Create a text splitter using NLTK

# https://python.langchain.com/docs/how_to/split_by_token/

from langchain.text_splitter import NLTKTextSplitter

def chunk_docs_nltk(documents: list, chunk_size=500, chunk_overlap=50) -> list:

    text_splitter = NLTKTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap)

    chunks_start = text_splitter.split_documents(documents)

    chunks_end = remove_empty_chunks(chunks_start=chunks_start)

    return chunks_end

#####

def test_chunk_docs_nltk(): 
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    chunks = chunk_docs_nltk(documents=docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_nltk()


Created a chunk of size 744, which is longer than the specified 500
Created a chunk of size 796, which is longer than the specified 500
Created a chunk of size 653, which is longer than the specified 500
Created a chunk of size 1349, which is longer than the specified 500
Created a chunk of size 742, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 580, which is longer than the specified 500
Created a chunk of size 750, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 541, which is longer than the specified 500
Created a chunk of size 1038, which is longer than the specified 500
Created a chunk of size 577, which is longer than the specified 500
Created a chunk of size 587, which is longer than the specified 500
Created a chunk of size 605, which is longer t

630
Medi-Cal
Member Handbook
Combined Evidence of Coverage
and Disclosure Form                   
 
CAHealthWellness.com
For TTY, contact California Relay by dialing 711 and 
provide the Member Services number: 1-877-658-0305


In [27]:
# Create a text splitter semantic chunking 

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/

from langchain_experimental.text_splitter import SemanticChunker

def chunk_docs_semantic(documents: list, ) -> list:

    # TODO - Use embeddings parameter
    text_splitter = SemanticChunker(create_embeddings_openai(), breakpoint_threshold_type="percentile")

    chunks_start = text_splitter.split_documents(documents)

    # Remove empty chunks
    chunks_end = remove_empty_chunks(chunks_start)

    return chunks_end

#####

def test_chunk_docs_semantic():
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    chunks = chunk_docs_semantic(docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_semantic()

252
Medi-Cal
Member Handbook
Combined Evidence of Coverage
and Disclosure Form                   
 
CAHealthWellness.com
For TTY, contact California Relay by dialing 711 and 
provide the Member Services number: 1-877-658-0305



In [28]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, collection_name: str, vector_size: int, embeddings: Embeddings, documents: list) -> QdrantVectorStore:

    # Initialize the Qdrant client
    qdrant_client = QdrantClient(location=location)

    # Create a collection in Qdrant
    qdrant_client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

    # Initialize QdrantVectorStore with the Qdrant client
    qdrant_vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name, embedding=embeddings)
    
    qdrant_vector_store.add_documents(documents)
    
    return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
    embeddings = create_embeddings_openai()
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    print(len(docs))
    chunks = chunk_docs_recursive(docs)
    print(len(chunks))
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    print(vector_store.collection_name)

test_create_qdrant_vector_store()

134
615
test


In [29]:
# Create a Qdrant retriever

# TODO - Add reference 

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

    retriever = vector_store.as_retriever()

    return retriever

#####

def test_create_retriever_qdrant(text: str = None):
    embeddings = create_embeddings_openai()
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    chunks = chunk_docs_recursive(docs)
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    retriever = create_retriever_qdrant(vector_store)
    if text:
        docs = retriever.invoke(text)
        print(docs[0])

print('\nQDRANT')
test_create_retriever_qdrant('What is my benefit for acupuncture?')


QDRANT
page_content='Acupuncture 
 
We will cover up to two outpatient acupuncture services in any calendar month.  Additional 
services may be covered if they are determined to be medically necessary.  You must receive 
prior authorization to receive more than two acupuncture services in a calendar month.   
 
There is no frequency limit if you are getting acupuncture services through the Early and 
Periodic Screening, Diagnosis, & Treatment program,    
 
Allergy Services' metadata={'source': 'docs/CHW_EOC_04-21-2017_ENG.pdf', 'file_path': 'docs/CHW_EOC_04-21-2017_ENG.pdf', 'page': 57, 'total_pages': 134, 'format': 'PDF 1.5', 'title': '', 'author': 'PTN Consulting', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2013', 'producer': 'Microsoft® Word 2013', 'creationDate': "D:20170421101839-07'00'", 'modDate': "D:20170421103401-07'00'", 'trapped': '', '_id': 'b3b7e1bbd16d446b9c841c4baa6c9119', '_collection_name': 'test'}


In [30]:
# Create a Vertex AI retriever

# https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search/

from langchain_google_community import VertexAISearchRetriever

def create_retriever_vertexai() -> VertexAISearchRetriever:

    retriever = VertexAISearchRetriever(project_id=os.environ['PROJECT_ID'], location_id=os.environ['LOCATION_ID'], data_store_id=os.environ['DATA_STORE_ID'], max_documents=3)

    return retriever

#####

def test_create_retriever_vertexai(text: str = None):
    retriever = create_retriever_vertexai()
    if text:
        docs = retriever.invoke(text)
        print(docs[0])

print('\nVERTEX AI')
test_create_retriever_vertexai('What is my benefit for acupuncture?')


VERTEX AI


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


page_content='# Member Handbook Combined Evidence of Coverage & Disclosure Form

california health & wellness

## Section 6: Your Benefits at a Glance

The chart below shows a quick summary of your benefits. This list does not include every covered and non-covered benefit or service. All services may have certain limits, which are described more fully in Section 7: Your Benefits and Services. Also please note that some services require prior approval by California Health & Wellness.

_START_OF_TABLE_
TABLE_IN_MARKDOWN:
|-|-|-|
| Service | Coverage | Details and Limitations |
| Abortion | Covered | Prior Authorization is not |
|  |  | required for outpatient |
|  |  | services. See Page 54 for |
|  |  | more details |
| Acupuncture | Covered | Up to 2 acupuncture services |
|  |  | in any calendar month. |
|  |  | Additional treatments may be |
|  |  | covered if medically necessary |
| Alcohol and Substance Abuse Treatment | Services (including drugs used for treatment and outpatient h

In [31]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

def create_chat_prompt_template(prompt: str = None) -> ChatPromptTemplate:
    
    template = '''
        You are a helpful conversational agent for the State of California.
    Your expertise is fully understanding the California Health & Wellness health plan. 
    You need to answer questions posed by the member, who is trying to get answers about their health plan.  
    Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

    You will be analyzing the health plan documents to derive a good answer, based on the following information:
    1. The question asked.
    2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

    Now it's your turn!

    {question}

    {context}
    '''
    
    prompt = PromptTemplate.from_template(template)

    return prompt

#####

def test_create_chat_prompt_template():
    prompt = create_chat_prompt_template()
    print(prompt)

test_create_chat_prompt_template()

input_variables=['context', 'question'] template="\n        You are a helpful conversational agent for the State of California.\n    Your expertise is fully understanding the California Health & Wellness health plan. \n    You need to answer questions posed by the member, who is trying to get answers about their health plan.  \n    Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. \n\n    You will be analyzing the health plan documents to derive a good answer, based on the following information:\n    1. The question asked.\n    2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.\n\n    Now it's your turn!\n\n    {question}\n\n    {context}\n    "


In [36]:
# Create a Langchain chain..

# https://python.langchain.com/docs/integrations/llms/google_ai/
# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://ai.google.dev/gemini-api/docs/safety-settings 

from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter

def create_chain (model_name: str, prompt: ChatPromptTemplate, retriever: BaseRetriever):

    if "gemini" in model_name.lower():
        llm = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=0,
            safety_settings={
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                },
            )
    else:
        print("Unsuported model name")
        
    chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
        | RunnablePassthrough.assign(context=itemgetter("context")) 
        | {"response": prompt | llm, "context": itemgetter("context")}
        )

    return chain

#####

def test_create_chain_qdrant():
    embeddings = create_embeddings_openai()
    docs = process_file('docs/CHW_EOC_04-21-2017_ENG.pdf')
    chunks = chunk_docs_recursive(docs)
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    retriever = create_retriever_qdrant(vector_store)
    chat_prompt_template = create_chat_prompt_template()
    chain = create_chain('gemini-1.5-flash', chat_prompt_template, retriever)
    result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
    print(result)

print('\nQDRANT')
test_create_chain_qdrant()

###

def test_create_chain_vertexai():
    retreiver = create_retriever_vertexai()
    chat_prompt_template = create_chat_prompt_template()
    chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)
    result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
    print(result)

print('\nVERTEX AI')
test_create_chain_vertexai()


QDRANT
{'response': AIMessage(content='California Health & Wellness covers up to two outpatient acupuncture services per calendar month.  If you need more than two services, you will need to get prior authorization.  There is no frequency limit if you are getting acupuncture services through the Early and Periodic Screening, Diagnosis, & Treatment program. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5f2c7621-8651-432f-aa24-36b359344934-0', usage_metadata={'input_tokens': 1448, 'output_tokens': 57, 'total_tokens': 1505}), 'cont

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'response': AIMessage(content='Your California Health & Wellness plan covers acupuncture. You are eligible for up to 2 acupuncture services in any calendar month.  Additional treatments may be covered if medically necessary.  For more details on coverage and limitations, please refer to Section 7: Your Benefits and Services in your Member Handbook. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3c3616ab-4654-4ef1-8b39-907c85f12c4c-0', usage_metadata={'input_tokens': 735, 'output_tokens': 59, 'total_tokens': 794}), 'context': [Doc

In [37]:
# Generate answers from a chain usin a list of questions

def generate_answers_contexts(chain, questions: list):
    
    answers = []
    contexts = []

    # Loop over the list of questions and call the chain to get the answer and context
    for question in questions:
        print(question)

        # Call the chain to get answers and contexts
        response = chain.invoke({"question" : question})
        print(response)
        
        # Capture the answer and context 
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])

    return answers, contexts

#####

In [38]:
# Run a Ragas evaluation 

from datasets import Dataset
from pandas import DataFrame
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, answer_correctness, context_recall, context_precision)

def run_ragas_evaluation(chain, questions: list, groundtruths: list, eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]):

  answers = []
  contexts = []
  answers, contexts = generate_answers_contexts(chain, questions)

  # Create the input dataset 
  input_dataset = Dataset.from_dict({
  "question" : questions,         # From the dataframe
  "answer" : answers,             # From the chain
  "contexts" : contexts,          # From the chain
  "ground_truth" : groundtruths   # From the dataframe
  })

  # Run the Ragas evaluation using the input dataset and eval metrics
  ragas_results = evaluate(input_dataset, eval_metrics)
  ragas_results_df = ragas_results.to_pandas()
  
  return ragas_results, ragas_results_df
  
#####

def test_run_ragas_evaluation():
  print("test")    

test_run_ragas_evaluation()

test


In [39]:
# Query Vertex AI Search datastore using HTTP Post

import requests
import google.auth
from google.auth.transport.requests import Request

credentials, project_id = google.auth.default()
credentials.refresh(Request())
access_token = credentials.token
print(access_token)

def query_chunks(query, n=5):
    
  PROJECT_ID = os.environ['PROJECT_ID']
  LOCATION_ID = os.environ['LOCATION_ID']
  DATA_STORE_ID = os.environ['DATA_STORE_ID']

  if LOCATION_ID == 'us':
    api_endpoint = 'us-discoveryengine.googleapis.com'
  else:
    api_endpoint = 'discoveryengine.googleapis.com'

  url = f"https://{api_endpoint}/v1alpha/projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search"
  print(url)
  
  headers = {
      "Authorization": f"Bearer {access_token}",
      "Content-Type": "application/json",
  }
  
  post_data = {
      "servingConfig": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search",
      "pageSize": n,
      "query": query,
      "contentSearchSpec": {"searchResultMode": "CHUNKS"},
  }
  
  response = requests.post(url, headers=headers, json=post_data)

  if response.status_code != 200:
    print(
        f"Error retrieving search results: {response.status_code} -"
        f" {response.text}"
    )

  return response.json()

#####

test = query_chunks('What is my benefit for acupuncture?')
print(test)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ya29.a0AcM612z2sbc44ug6ZTSx6YksPynDiCIRWydpZjUAuk8WBmPgX2DNO068CxUsiQPloVjSPuOhClc5foI2jNCbcvt_2j15GeqRkovZVktxwxpzM77cpXjxKQ2fW96q8pNZOnwvavpmHX-LzW0c__ObmCgXaP871ta73NO1fqDeCAaCgYKAV0SARESFQHGX2Mif0ZoZU1PxyQvrkeuoyY85A0177
https://us-discoveryengine.googleapis.com/v1alpha/projects/aie4-demo/locations/us/collections/default_collection/dataStores/aie4-demo-datastore_1729262666131/servingConfigs/default_search:search
{'results': [{'chunk': {'name': 'projects/395640738565/locations/us/collections/default_collection/dataStores/aie4-demo-datastore_1729262666131/branches/0/documents/17f8562d9e55f76698d75fa2e258e2a5/chunks/c40', 'id': 'c40', 'content': "# Member Handbook Combined Evidence of Coverage & Disclosure Form\n\ncalifornia health & wellness\n\n## Section 6: Your Benefits at a Glance\n\nThe chart below shows a quick summary of your benefits. This list does not include every covered and non-covered benefit or service. All services may have certain limits, which are described more full

# Create RAG chain using Google Vertex AI

In [40]:
# Build RAG chain using Vertex AI Agent Builder datastore

retreiver = create_retriever_vertexai()
chat_prompt_template = create_chat_prompt_template()
chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
# Test the chain 

questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What are the responsibilities of my PCP?',]

for question in questions:
    print(question)
    result = chain.invoke({"question" : question})
    print(result)
    print(result["response"].content)

What is my benefit for acupuncture?
{'response': AIMessage(content='Your California Health & Wellness plan covers acupuncture. You are eligible for up to 2 acupuncture services in any calendar month.  Additional treatments may be covered if medically necessary.  For more details on coverage and limitations, please refer to Section 7: Your Benefits and Services in your Member Handbook. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-e5d96cfa-15dd-455f-b293-0dfb0cb3f6d1-0', usage_metadata={'input_tokens': 735, 'output_tokens': 59, 't

# Create synthetic testset

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.testset.generator import TestsetGenerator

# Load the docs
documents = []

paths = ['docs/CHW_EOC_04-21-2017_ENG.pdf']

for path in paths:
    documents.extend(process_file(path=path))

# Chunk the docs 
# chunks = chunk_docs_nltk(documents, 1500, 150)
chunks = chunk_docs_semantic(documents)

# Set up the parameters for generating the testset
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
generator = TestsetGenerator.from_langchain(generator_llm, critic_llm, embeddings)
distributions = {simple: 0.5, multi_context: 0.4, reasoning: 0.1}

# Generate the testset and save to disk 
testset = generator.generate_with_langchain_docs(documents=chunks, test_size=50, distributions=distributions)
testset_df = testset.to_pandas()

print("Writing synthetic_testset.csv")
testset_df.to_csv("testsets/synthetic_testset.csv")
testset_df

# Evaluate synthetic dataset using Ragas

In [ ]:
# Evaluate the synthetic testset using Ragas

import pandas as pd

# Run the Ragas evaluation and show the results
# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/synthetic_testset.csv")

questions = testset_df["question"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["ground_truth"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]  

# Specify the eval metrics
eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]

# Run the Ragas evaluation and show the results
ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)

# Write the results to disk
print("Writing synthetic_testset_ragas_results.csv")
ragas_results_df.to_csv("ragas/synthetic_testset_ragas_results.csv")

# Show the resutls
ragas_results

# Create sample ReAct agent using Gemini

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            safety_settings={
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                },
            )

tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, 
                           tools, 
                           prompt)

agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True)

agent_executor.invoke(
    {
        "input": "What's the paper 1605.08386 about?",
        }
)

# Use Unstructured to parse PDFs

In [53]:
%pip install -qU "unstructured-ingest[pdf]" unstructured faiss-cpu

I0000 00:00:1729598196.665302 3894370 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [43]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

In [44]:
import os 

directory_with_pdfs="content/data"
directory_with_results="content/output"

Pipeline.from_configs(
    context=ProcessorConfig(),
    indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
    downloader_config=LocalDownloaderConfig(),
    source_connection_config=LocalConnectionConfig(),
    partitioner_config=PartitionerConfig(
        partition_by_api=True,
        api_key=os.getenv("UNSTRUCTURED_API_KEY"),
        partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
        strategy="hi_res",
        additional_partition_args={
            "split_pdf_page": True,
            "split_pdf_concurrency_level": 15,
            },
        ),
    uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
).run()

2024-10-22 07:53:52,446 MainProcess INFO     created index with configs: {"input_path": "content/data", "recursive": false}, connection configs: {"access_config": "**********"}
2024-10-22 07:53:52,448 MainProcess INFO     Created download with configs: {"download_dir": null}, connection configs: {"access_config": "**********"}
2024-10-22 07:53:52,449 MainProcess INFO     created partition with configs: {"strategy": "hi_res", "ocr_languages": null, "encoding": null, "additional_partition_args": {"split_pdf_page": true, "split_pdf_concurrency_level": 15}, "skip_infer_table_types": null, "fields_include": ["element_id", "text", "type", "metadata", "embeddings"], "flatten_metadata": false, "metadata_exclude": [], "metadata_include": [], "partition_endpoint": null, "partition_by_api": true, "api_key": "*******", "hi_res_model_name": null}
2024-10-22 07:53:52,450 MainProcess INFO     Created upload with configs: {"output_dir": "content/output"}, connection configs: {"access_config": "*******

In [45]:
from unstructured.staging.base import elements_from_json

def load_processed_files(directory_path):
    elements = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                elements.extend(elements_from_json(filename=file_path))
            except IOError:
                print(f"Error: Could not read file {filename}.")

    return elements

elements = load_processed_files(directory_with_results)

I0000 00:00:1729598040.630811 3894370 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
from langchain_core.documents import Document

documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

In [56]:
# embeddings = create_embeddings_vertexai() 
# vector_store = create_qdrant_vector_store(":memory:", "test", 768, embeddings, documents)

from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [57]:
chain = create_chain('gemini-1.5-flash', chat_prompt_template, retriever)

In [58]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What are the responsibilities of my PCP?',]

for question in questions:
    print(question)
    result = chain.invoke({"question" : question})
    print(result)
    print(result["response"].content)

What is my benefit for acupuncture?
{'response': AIMessage(content="The California Health & Wellness plan covers up to two outpatient acupuncture services in any calendar month. If you need more than two acupuncture services in a month, you will need to get prior authorization from the plan.  This means you'll need to have your doctor submit a request to the plan explaining why you need more than two acupuncture services. \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-49cea10e-14fc-460e-85d2-e4d32c5378e6-0', usage_metadata={'input